In [ ]:
# sign_capture.py
import cv2
import mediapipe as mp
import csv
import os
import numpy as np
from collections import deque

# ---------------------------
# CONFIGURATION (Sign Sequence)
# ---------------------------
SIGN_LABEL      = "q"
SEQUENCE_LENGTH = 30
POSE_LM         = 33
HAND_LM         = 21
SIGN_LM_TOTAL   = POSE_LM + 2 * HAND_LM       # 75 landmarks
SIGN_FEAT_LEN   = SIGN_LM_TOTAL * 4           # 300 values/frame
SIGN_CSV        = "sign_sequences.csv"

# ---------------------------
# HEADER CREATION
# ---------------------------
def write_header(path, lm_total, seq_len):
    if os.path.exists(path) and os.stat(path).st_size > 0:
        return
    header = ["class"]
    for frame_idx in range(1, seq_len+1):
        for lm_idx in range(1, lm_total+1):
            header += [
                f"f{frame_idx}_lm{lm_idx}_x",
                f"f{frame_idx}_lm{lm_idx}_y",
                f"f{frame_idx}_lm{lm_idx}_z",
                f"f{frame_idx}_lm{lm_idx}_v",
            ]
    with open(path, "w", newline="") as f:
        csv.writer(f).writerow(header)
    print(f"Header written to {path}")

write_header(SIGN_CSV, SIGN_LM_TOTAL, SEQUENCE_LENGTH)

# ---------------------------
# LANDMARK EXTRACTION
# ---------------------------
def extract_landmarks(lm_list, expected_count):
    if not lm_list:
        return [0.0] * expected_count * 4
    vals = []
    for lm in lm_list.landmark:
        x = lm.x if lm.x==lm.x else 0.0
        y = lm.y if lm.y==lm.y else 0.0
        z = lm.z if lm.z==lm.z else 0.0
        v = lm.visibility if lm.visibility==lm.visibility else 0.0
        vals += [x, y, z, v]
    if len(vals) < expected_count*4:
        vals += [0.0] * (expected_count*4 - len(vals))
    return vals

# ---------------------------
# CAPTURE LOOP
# ---------------------------
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

buffer = deque(maxlen=SEQUENCE_LENGTH)
seq_count = 0
countof = 0

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as holistic:
    print("Press 'q' to quit sign capture.")
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = holistic.process(img_rgb)

        # Draw landmarks on frame
        if res.pose_landmarks:
            mp_drawing.draw_landmarks(frame, res.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        if res.left_hand_landmarks:
            mp_drawing.draw_landmarks(frame, res.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        if res.right_hand_landmarks:
            mp_drawing.draw_landmarks(frame, res.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Overlay coordinates for pose landmarks
        if res.pose_landmarks:
            for idx, lm in enumerate(res.pose_landmarks.landmark):
                cx, cy = int(lm.x * w), int(lm.y * h)
                coord_text = f"{idx}:({lm.x:.2f},{lm.y:.2f})"
                cv2.putText(frame, coord_text, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX,
                            0.4, (255, 255, 255), 1, cv2.LINE_AA)

        # Extract features and buffer them
        p = extract_landmarks(res.pose_landmarks, POSE_LM)
        lh = extract_landmarks(res.left_hand_landmarks, HAND_LM)
        rh = extract_landmarks(res.right_hand_landmarks, HAND_LM)
        feats = p + lh + rh

        buffer.append(feats)
        if len(buffer) == SEQUENCE_LENGTH:
            countof += 1
            seq_count += 1
            row = [SIGN_LABEL] + [v for frame_feats in buffer for v in frame_feats]
            with open(SIGN_CSV, "a", newline="") as f:
                csv.writer(f).writerow(row)
            print(f"[Sign Seq {seq_count}] saved.")
            buffer.clear()

        # Display buffer and count
        cv2.putText(frame, f"Buf: {len(buffer)}/{SEQUENCE_LENGTH}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(frame, f"Count: {countof}", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Sign Capture", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
